# Optimiaing Model Parameters

Training a model is an iterative process; 
in each iteration(called an epoch) the model
- makes a guess about the output, 
- calculates the error in its guess(loss)
- collects the derivatives of the error with respect to its parameters
- optimizes the parameters using gradient descent

# Prerequisite Code

In [ ]:
# Datasets & DataLoaders, Build Model에서 사용한 코드
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

# Hyperparameterse

하이퍼파라미터는 변경 가능한 파라미터로 모델의 최적화를 위해 조정할 수 있다. 여러가지 하이퍼 파라미터 값들은 모델 학습과 convergence rates에 영향을 줄 수 있다.

- rate of convergence: a measure of how the difference between the solution point and its estimates goes to zero

학습을 위해 다음의 하이퍼 파라미터들을 튜닝한다:
- Number of Epochs
  - the number times to iterate over the dataset
- Batch size
  - the number of data samples propagated through the network before the parameters are updated
- Learnng Rate
  - how much to update models parameters at each batch/epoch. 값이 작을 수록 느린 학습

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimiaztion Loop

하이퍼 파라미터를 설정하면 optimization loop을 사용해 모델을 학습, 최적화시킬 수 있다.
각각의 iteration == epoch

하나의 epoch는 두 가지 메인 파트로 구성:
- The Train Loop: iterate over the training and try to converge to optimal parametes
- The Validation/Test Loop: iterate over the test dataset to check if model performance is improving

## Loss Function

## Optimizer

Optimization
- the process of adjusting model parameters to reduce modle error in each training step

Optimization algorithms으로 어떻게 최적화를 할 것인지 정의

All optimization logic is encapulated in the optimizer object

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
# PyTorch에서 ADAM, RMSProp 같은 다른 optimizers 사용가능

# Full Implementation

In [ ]:
# full implementation of train_loop 

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation - Optimization process
        optimizer.zero_grad() # to rest the gradients of model parameters
        loss.backward() # backpropagate the prediction loass with a call to loss.backward()
        optimizer.step() # to adject the parameters by the gradient collected in the backward pass

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# initialize the loss function and optimzer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# pass the loss function and optimzer to train_loop and test_loop
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298863  [    0/60000]
loss: 2.286261  [ 6400/60000]
loss: 2.269766  [12800/60000]
loss: 2.264961  [19200/60000]
loss: 2.252468  [25600/60000]
loss: 2.223565  [32000/60000]
loss: 2.229445  [38400/60000]
loss: 2.197865  [44800/60000]
loss: 2.195070  [51200/60000]
loss: 2.167722  [57600/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 2.160815 

Epoch 2
-------------------------------
loss: 2.168206  [    0/60000]
loss: 2.157894  [ 6400/60000]
loss: 2.105493  [12800/60000]
loss: 2.121247  [19200/60000]
loss: 2.072092  [25600/60000]
loss: 2.014868  [32000/60000]
loss: 2.042094  [38400/60000]
loss: 1.966685  [44800/60000]
loss: 1.970884  [51200/60000]
loss: 1.899888  [57600/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.894716 

Epoch 3
-------------------------------
loss: 1.930654  [    0/60000]
loss: 1.895056  [ 6400/60000]
loss: 1.783226  [12800/60000]
loss: 1.820130  [19200/60000]
loss: 1.710875  [25600/60000]
loss: 1.664680  [32000/600